In [2]:
from google.colab import drive
from numpy.ma.core import asarray

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/chest_xray/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
data_path = 'data/chest_xray'

Sto considerando di usare i seguenti modelli di ML:
1. ResNet.
2. GoogLeNet.
3. AlexNet.

In [3]:
import os
import pathlib

data_dir = pathlib.Path(os.path.join(data_path, 'train'))
total = len(list(data_dir.glob('**/*.jpeg')))
bacteria = len(list(data_dir.glob('**/BACTERIA-*.jpeg')))
virus = len(list(data_dir.glob('**/VIRUS-*.jpeg')))
P = len(list(data_dir.glob('**/NORMAL-*.jpeg')))
N = total - P
print(f'#training set = {total}')
print(f'#positives = {P}')
print(f'#negatives = {N} (bacteria = {bacteria}, virus = {virus})')

#training set = 5232
#positives = 1349
#negatives = 3883 (bacteria = 2538, virus = 1345)


In [9]:
import keras

# see: https://keras.io/api/data_loading/image/#image_dataset_from_directory-function
(training, validation) = keras.utils.image_dataset_from_directory(
    data_dir,
    color_mode='grayscale',
    image_size=(224, 224),
    seed=42,
    validation_split=0.2,
    subset='both'
)

Found 5232 files belonging to 2 classes.
Using 4186 files for training.
Using 1046 files for validation.


In [10]:
help(training.batch)

Help on method batch in module tensorflow.python.data.ops.dataset_ops:

batch(batch_size, drop_remainder=False, num_parallel_calls=None, deterministic=None, name=None) -> 'DatasetV2' method of tensorflow.python.data.ops.prefetch_op._PrefetchDataset instance
    Combines consecutive elements of this dataset into batches.

    >>> dataset = tf.data.Dataset.range(8)
    >>> dataset = dataset.batch(3)
    >>> list(dataset.as_numpy_iterator())
    [array([0, 1, 2]), array([3, 4, 5]), array([6, 7])]

    >>> dataset = tf.data.Dataset.range(8)
    >>> dataset = dataset.batch(3, drop_remainder=True)
    >>> list(dataset.as_numpy_iterator())
    [array([0, 1, 2]), array([3, 4, 5])]

    The components of the resulting element will have an additional outer
    dimension, which will be `batch_size` (or `N % batch_size` for the last
    element if `batch_size` does not divide the number of input elements `N`
    evenly and `drop_remainder` is `False`). If your program depends on the
    batches ha

In [13]:
batch = training.batch(100)

In [14]:
numpy = list(batch.as_numpy_iterator())

2025-01-26 20:17:03.055443: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 642252800 exceeds 10% of free system memory.
2025-01-26 20:17:03.154102: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 642265600 bytes after encountering the first element of size 642265600 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2025-01-26 20:17:03.385702: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 199098368 exceeds 10% of free system memory.


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot batch tensors with different shapes in component 0. First element had shape [32,224,224,1] and element 30 had shape [26,224,224,1]. [Op:IteratorGetNext] name: 

Le immagini nel training set sono 5232 di cui: 1349 appartenenti alla classe NORMAL mentre 3883 alla classe PNEUMONIA.

```py
#training set = 5232
#positives = 1349
#negatives = 3883 (bacteria = 2538, virus = 1345)
```

Sto considerando diverse strategie per aumentare la dimensione del dataset:
1. Trasformazioni geometriche: specchiare, ruotare, tagliare, allungare.
   a. Specchiare e ruotare non mi sembrano che possano essere molto utili per questa tipologia di problema. Similmente anche
      il ritagliare l'immagine potrebbe rimuovere porzioni di immagini che potrebbero rivelarsi fondamentali per individuare o meno
      la polmonite.
   b. Tag
2. Modificare la luminosità.

È necessario ridimensionare le immagini per:
- Utilizzare meno risorse computazionali.
- Velocizzare sia addestramento che inferenza.
Al momento sto considerando le diverse strategie per il ridimensionamento delle immagini:
1. Ridimensionare tutte le immagini a 133x133 dove 133 è l'altezza minima tra tutte le immagini.
2. Ridimensionare tutte le immagini a 290x290 dove 290 è l'altezza media delle immagini

In [3]:
from keras import activations, layers, models
from typing import Tuple, Callable


class ResBlock(layers.Layer):
    def __init__(self, k: Tuple[int, int], n: int, s: Tuple[int, int], **kwargs):
        super().__init__(**kwargs)
        self.conv1 = layers.Conv2D(n, k, s)
        self.bn1   = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(n, k, s)
        self.bn2   = layers.BatchNormalization()
        self.sum   = layers.Add()

    def __call__(self, x):
        z = self.conv1(x)
        z = self.bn1(z)
        z = activations.leaky_relu(z, 0.2)
        z = self.conv2(z)
        z = self.bn2(z)
        return self.sum(x, z)


class Resizer(layers.Layer):
    def __init__(self, target: Tuple[int, int], interpolation='bilinear', r: int=4, **kwargs):
        super().__init__(**kwargs)
        ## learnable layers
        self.conv1 = layers.Conv2D(16, (7, 7), activation=activations.leaky_relu)
        self.conv2 = layers.Conv2D(16, (1, 1), activation=activations.leaky_relu)
        self.bn1   = layers.BatchNormalization()
        self.r_blocks = [ResBlock((3, 3), 16, (1, 1)) for _ in range(r)]
        self.conv3 = layers.Conv2D(16, (3, 3), (1, 1))
        self.bn2   = layers.BatchNormalization()
        self.conv4 = layers.Conv2D(3, (7, 7), (1, 1))
        ## non-learnable layers
        self.sum  = layers.Add()
        self.sampler = layers.UpSampling2D(target, interpolation=interpolation)

    def __call__(self, x):
        y = self.conv1(x)
        y = self.conv2(y)
        y = self.bn1(y)
        y = self.sampler(y)
        z = y
        for r_block in self.r_blocks:
            z = r_block(z)
        z = self.conv3(z)
        z = self.bn2(z)
        z = self.sum(y, z)
        z = self.conv4(z)
        x = self.sampler(x)
        return self.sum(x, z)


2025-01-21 16:46:49.252417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-21 16:46:49.272130: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-21 16:46:49.278932: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 16:46:49.338871: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from keras import activations, layers, models


def alex_net():
    model = models.Sequential([
        layers.Conv2D(96, (11, 11), (4, 4),
                      activation=activations.relu),
        layers.BatchNormalization(),
        layers.MaxPool2D((3, 3), (2, 2)),
        layers.Conv2D(256, (5, 5), (2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.MaxPool2D((3, 3), (2, 2)),
        layers.Conv2D(384, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(384, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.MaxPool2D((3, 3), (2, 2)),
        layers.Flatten(),
        layers.Dense(4096),
        layers.Dropout(0.5),
        layers.Dense(4096),
        layers.Dropout(0.5),
        layers.Dense(2, activation=activations.softmax)
    ])
    return model

In [7]:
from keras import callbacks

model = alex_net()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
hist = model.fit(training, epochs=30, validation_data=validation, callbacks=[callbacks.EarlyStopping(patience=10)])

Epoch 1/30
  8/131 ━━━━━━━━━━━━━━━━━━━━ 28:04 14s/step - accuracy: 0.5177 - loss: 61.4990